In [1]:
import pandas as pd
import getpass
import zipfile
import os
import time
import datetime
import threading
import pyzipper


class Zipndel:
    def __init__(self, file_name: str = 'df', file_format: str = 'csv', self_destruct_time=None):
        """
        Initialize Zipndel object.

        Parameters:
        file_name (str): the name of the file to be written, default is 'df'
        file_format (str): the file format of the file to be written, default is 'csv'
        self_destruct_time (tuple): a tuple of (hours, minutes, seconds) until self-destruct, default is None
        """
        self.file_name = file_name
        self.file_format = file_format
        self.self_destruct_time = self_destruct_time

    def zipit(self, df: pd.DataFrame) -> None:
        """
        Write the input dataframe to a file, create a zip file with the written file, set a password for the zip file,
        and delete the written file.

        Parameters:
        df (pd.DataFrame): the input dataframe to be written to file and zipped

        Example:
        >>> df = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6], 'C': [7, 8, 9]})
        >>> Zipndel().zipit(df)
        """
        # write dataframe to a pandas supported file
        write_func = getattr(df, f'to_{self.file_format}')
        write_func(self.file_name, index=False)

        # create zip file and add the written file to it
        df_zip = f"{self.file_name}.zip"
        with pyzipper.AESZipFile(df_zip, 'w', compression=pyzipper.ZIP_DEFLATED, encryption=pyzipper.WZ_AES) as zf:
            zf.setpassword(bytes(getpass.getpass('Password:'), 'utf-8'))
            zf.write(self.file_name)

        # delete written file
        os.remove(self.file_name)

        # self-destruct timer
        if self.self_destruct_time is not None:
            t = threading.Thread(target=self.self_destruct,
                                 args=self.self_destruct_time)
            t.start()

    def self_destruct(self, hours: int, minutes: int, seconds: int) -> None:
        """
        Set the self-destruct timer for the zip file.

        Parameters:
        hours (int): number of hours until self-destruct
        minutes (int): number of minutes until self-destruct
        seconds (int): number of seconds until self-destruct
        """
        df_zip = f"{self.file_name}.zip"
        self_destruct_time = time.time() + hours * 60 * 60 + minutes * 60 + seconds
        while True:
            if time.time() > self_destruct_time:
                os.remove(df_zip)
                print(
                    f"{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - Zip file deleted due to self-destruct timer.")
                break
            time.sleep(5)


# create an instance of Unzipndel
unzipper = Unzipndel(file_name='test_file', file_format='csv')

# prompt for password
unzipper.set_password()

# extract the dataframe from the zip file
df = unzipper.unzipit()

# display the dataframe
print(df)


NameError: name 'Unzipndel' is not defined

In [2]:
import os
import pyzipper
import pandas as pd

class Unzipndel:
    def __init__(self, file_name: str = 'df', file_format: str = 'csv', password=None):
        """
        Initialize Unzipndel object.

        Parameters:
        file_name (str): the name of the zip file to be extracted, default is 'df'
        file_format (str): the file format of the file to be extracted, default is 'csv'
        password (str): the password for the zip file, default is None
        """
        self.file_name = file_name
        self.file_format = file_format
        self.password = password
        
    def unzipit(self) -> pd.DataFrame:
        """
        Extract a zip file, read the contents of the extracted file into a pandas dataframe, delete the extracted file,
        and return the dataframe.

        Returns:
        pd.DataFrame: the extracted dataframe

        Example:
        >>> unzipper = Unzipndel(password='mypassword')
        >>> df = unzipper.unzipit()
        """
        # extract the zip file
        df_zip = f"{self.file_name}.zip"
        with pyzipper.AESZipFile(df_zip, 'r') as zf:
            if self.password is not None:
                zf.setpassword(bytes(self.password, 'utf-8'))
            zf.extractall()

        # read the extracted file into a pandas dataframe
        df = pd.read_csv(f"{self.file_name}.{self.file_format}")

        # delete the extracted file
        os.remove(f"{self.file_name}.{self.file_format}")

        return df


In [4]:
import os
import pyzipper
import pandas as pd
import time
import threading
import datetime

class Zipndel:
    def __init__(self, file_name: str = 'df', file_format: str = 'csv', self_destruct_time=None, password=None):
        """
        Initialize Zipndel object.

        Parameters:
        file_name (str): the name of the file to be written, default is 'df'
        file_format (str): the file format of the file to be written, default is 'csv'
        self_destruct_time (tuple): a tuple of (hours, minutes, seconds) until self-destruct, default is None
        password (str): the password for the zip file, default is None
        """
        self.file_name = file_name
        self.file_format = file_format
        self.self_destruct_time = self_destruct_time
        self.password = password

    def zipit(self, df: pd.DataFrame) -> None:
        """
        Write the input dataframe to a file, create a zip file with the written file, set a password for the zip file,
        and delete the written file.

        Parameters:
        df (pd.DataFrame): the input dataframe to be written to file and zipped

        Example:
        >>> df = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6], 'C': [7, 8, 9]})
        >>> Zipndel(password='mypassword').zipit(df)
        """
        # write dataframe to a pandas supported file
        write_func = getattr(df, f'to_{self.file_format}')
        write_func(self.file_name, index=False)

        # create zip file and add the written file to it
        df_zip = f"{self.file_name}.zip"
        with pyzipper.AESZipFile(df_zip, 'w', compression=pyzipper.ZIP_DEFLATED, encryption=pyzipper.WZ_AES) as zf:
            zf.setpassword(bytes(self.password, 'utf-8'))
            zf.write(self.file_name)

        # delete written file
        os.remove(self.file_name)

        # self-destruct timer
        if self.self_destruct_time is not None:
            t = threading.Thread(target=self.self_destruct, args=self.self_destruct_time)
            t.start()

    def self_destruct(self, hours: int, minutes: int, seconds: int) -> None:
        """
        Set the self-destruct timer for the zip file.

        Parameters:
        hours (int): number of hours until self-destruct
        minutes (int): number of minutes until self-destruct
        seconds (int): number of seconds until self-destruct
        """
        df_zip = f"{self.file_name}.zip"
        self_destruct_time = time.time() + hours * 60 * 60 + minutes * 60 + seconds
        while True:
            if time.time() > self_destruct_time:
                os.remove(df_zip)
                print(
                    f"{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - Zip file deleted due to self-destruct timer.")
                break
            time.sleep(5)


In [5]:
# create a sample dataframe
df = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6], 'C': [7, 8, 9]})

# create a Zipndel object with password protection and self-destruct timer
zipper = Zipndel(file_name='test_file', file_format='csv', password='mypassword', self_destruct_time=(0, 3, 0))

# zip the dataframe with password protection and self-destruct timer
zipper.zipit(df)

2023-03-06 00:47:33 - Zip file deleted due to self-destruct timer.


In [ ]:
didnt get any password to prompt when I ran: 
# create a Zipndel object with password protection and self-destruct timer
zipper = Zipndel(file_name='test_file', file_format='csv', password='mypassword', self_destruct_time=(0, 3, 0))

# zip the dataframe with password protection and self-destruct timer
zipper.zipit(df)

In [8]:
class Zipndel:
    def __init__(self, file_name: str = 'df', file_format: str = 'csv', self_destruct_time=None, password=None):
        """
        Initialize Zipndel object.
        
        Parameters:
        file_name (str): the name of the file to be written, default is 'df'
        file_format (str): the file format of the file to be written, default is 'csv'
        self_destruct_time (tuple): a tuple of (hours, minutes, seconds) until self-destruct, default is None
        password (str): the password for the zip file, default is None
        """
        self.file_name = file_name
        self.file_format = file_format
        self.self_destruct_time = self_destruct_time
        
        # prompt user for password if not provided in init
        if password is None:
            self.password = getpass(prompt='Enter password for zip file: ')
        else:
            self.password = password

    def zipit(self, df: pd.DataFrame) -> None:
        """
        Write the input dataframe to a file, create a zip file with the written file, set a password for the zip file,
        and delete the written file.
        
        Parameters:
        df (pd.DataFrame): the input dataframe to be written to file and zipped
        
        Example:
        >>> df = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6], 'C': [7, 8, 9]})
        >>> Zipndel(password='mypassword').zipit(df)
        """
        # write dataframe to a pandas supported file
        write_func = getattr(df, f'to_{self.file_format}')
        write_func(self.file_name, index=False)

        # create zip file and add the written file to it
        df_zip = f"{self.file_name}.zip"
        with pyzipper.AESZipFile(df_zip, 'w', compression=pyzipper.ZIP_DEFLATED, encryption=pyzipper.WZ_AES) as zf:
            zf.setpassword(bytes(self.password, 'utf-8'))
            zf.write(self.file_name)

        # delete written file
        os.remove(self.file_name)

        # self-destruct timer
        if self.self_destruct_time is not None:
            t = threading.Thread(target=self.self_destruct, args=self.self_destruct_time)
            t.start()

    def self_destruct(self, hours: int, minutes: int, seconds: int) -> None:
        """
        Set the self-destruct timer for the zip file.
        
        Parameters:
        hours (int): number of hours until self-destruct
        minutes (int): number of minutes until self-destruct
        seconds (int): number of seconds until self-destruct
        """
        df_zip = f"{self.file_name}.zip"
        self_destruct_time = time.time() + hours * 60 * 60 + minutes * 60 + seconds
        while True:
            if time.time() > self_destruct_time:
                os.remove(df_zip)
                print(
                    f"{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - Zip file deleted due to self-destruct timer.")
                break
            time.sleep(5)

            
# create a Zipndel object with password protection and self-destruct timer
zipper = Zipndel(file_name='test_file', file_format='csv', self_destruct_time=(0, 3, 0))


TypeError: 'module' object is not callable

In [9]:
import os
import pyzipper
import pandas as pd
import time
import threading
import datetime
from getpass import getpass

class Zipndel:
    def __init__(self, file_name: str = 'df', file_format: str = 'csv', self_destruct_time=None):
        """
        Initialize Zipndel object.

        Parameters:
        file_name (str): the name of the file to be written, default is 'df'
        file_format (str): the file format of the file to be written, default is 'csv'
        self_destruct_time (tuple): a tuple of (hours, minutes, seconds) until self-destruct, default is None
        """
        self.file_name = file_name
        self.file_format = file_format
        self.self_destruct_time = self_destruct_time

        # prompt user for password if not provided in init
        self.password = getpass(prompt='Enter password for zip file: ')

    def zipit(self, df: pd.DataFrame) -> None:
        """
        Write the input dataframe to a file, create a zip file with the written file, set a password for the zip file,
        and delete the written file.

        Parameters:
        df (pd.DataFrame): the input dataframe to be written to file and zipped

        Example:
        >>> df = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6], 'C': [7, 8, 9]})
        >>> Zipndel().zipit(df)
        """
        # write dataframe to a pandas supported file
        write_func = getattr(df, f'to_{self.file_format}')
        write_func(self.file_name, index=False)

        # create zip file and add the written file to it
        df_zip = f"{self.file_name}.zip"
        with pyzipper.AESZipFile(df_zip, 'w', compression=pyzipper.ZIP_DEFLATED, encryption=pyzipper.WZ_AES) as zf:
            zf.setpassword(bytes(self.password, 'utf-8'))
            zf.write(self.file_name)

        # delete written file
        os.remove(self.file_name)

        # self-destruct timer
        if self.self_destruct_time is not None:
            t = threading.Thread(target=self.self_destruct, args=self.self_destruct_time)
            t.start()

    def self_destruct(self, hours: int, minutes: int, seconds: int) -> None:
        """
        Set the self-destruct timer for the zip file.

        Parameters:
        hours (int): number of hours until self-destruct
        minutes (int): number of minutes until self-destruct
        seconds (int): number of seconds until self-destruct
        """
        df_zip = f"{self.file_name}.zip"
        self_destruct_time = time.time() + hours * 60 * 60 + minutes * 60 + seconds
        while True:
            if time.time() > self_destruct_time:
                os.remove(df_zip)
                print(
                    f"{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - Zip file deleted due to self-destruct timer.")
                break
            time.sleep(5)


In [10]:
# create a Zipndel object with password protection and self-destruct timer
zipper = Zipndel(file_name='test_file', file_format='csv', password='mypassword', self_destruct_time=(0, 3, 0))

# zip the dataframe with password protection and self-destruct timer
zipper.zipit(df)


TypeError: __init__() got an unexpected keyword argument 'password'

In [ ]:
import os
import pyzipper
import pandas as pd
from getpass import getpass

class Unzipndel:
    def __init__(self, file_name: str = 'df', file_format: str = 'csv'):
        """
        Initialize Unzipndel object.

        Parameters:
        file_name (str): the name of the zip file to be extracted, default is 'df'
        file_format (str): the file format of the file to be extracted, default is 'csv'
        """
        self.file_name = file_name
        self.file_format = file_format
        
    def unzipit(self) -> pd.DataFrame:
        """
        Extract a file from a password-protected zip file and read it into a pandas dataframe.

        Returns:
        pd.DataFrame: the extracted dataframe

        Example:
        >>> unzipper = Unzipndel(file_name='test_file', file_format='csv')
        >>> df = unzipper.unzipit()
        """
        # prompt user for password
        password = getpass(prompt='Enter password for zip file: ')

        # read the zip file
        with pyzipper.AESZipFile(f"{self.file_name}.zip") as zf:
            try:
                # use password to decrypt the zip file
                zf.setpassword(bytes(password, 'utf-8'))

                # get the names of all files in the zip file
                zip_files = zf.namelist()

                # check if the zip file contains the specified file
                if f"{self.file_name}.{self.file_format}" not in zip_files:
                    raise ValueError(f"The zip file does not contain {self.file_name}.{self.file_format}")

                # extract the specified file
                zf.extract(f"{self.file_name}.{self.file_format}")

                # read the extracted file into a pandas dataframe
                df = pd.read_csv(f"{self.file_name}.{self.file_format}")

                # delete the extracted file
                os.remove(f"{self.file_name}.{self.file_format}")

                # return the dataframe
                return df

            except ValueError as e:
                print(e)
                return None


In [21]:
import pyzipper
import pandas as pd
import time
import threading
import datetime
from getpass import getpass

class Zipndel:
    def __init__(self, file_name: str = 'df', file_format: str = 'csv', self_destruct_time=None, password=None):
        """
        Initialize Zipndel object.

        Parameters:
        file_name (str): the name of the file to be written, default is 'df'
        file_format (str): the file format of the file to be written, default is 'csv'
        self_destruct_time (tuple): a tuple of (hours, minutes, seconds) until self-destruct, default is None
        password (str): the password for the zip file, default is None
        """
        self.file_name = file_name
        self.file_format = file_format
        self.self_destruct_time = self_destruct_time

        # prompt user for password if not provided in init
        if password is None:
            self.password = getpass(prompt='Enter password for zip file: ')
        else:
            self.password = password

    def zipit(self, df: pd.DataFrame) -> None:
        """
        Write the input dataframe to a file, create a zip file with the written file, set a password for the zip file,
        and delete the written file.

        Parameters:
        df (pd.DataFrame): the input dataframe to be written to file and zipped

        Example:
        >>> df = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6], 'C': [7, 8, 9]})
        >>> Zipndel(password='mypassword').zipit(df)
        """
        # write dataframe to a pandas supported file
        write_func = getattr(df, f'to_{self.file_format}')
        write_func(self.file_name, index=False)

        # create zip file and add the written file to it
        df_zip = f"{self.file_name}.zip"
        with pyzipper.AESZipFile(df_zip, 'w', compression=pyzipper.ZIP_DEFLATED, encryption=pyzipper.WZ_AES) as zf:
            zf.setpassword(bytes(self.password, 'utf-8'))
            zf.write(self.file_name)

        # delete written file
        os.remove(self.file_name)

        # self-destruct timer
        if self.self_destruct_time is not None:
            t = threading.Thread(target=self.self_destruct, args=self.self_destruct_time)
            t.start()

    def self_destruct(self, hours: int, minutes: int, seconds: int) -> None:
        """
        Set the self-destruct timer for the zip file.

        Parameters:
        hours (int): number of hours until self-destruct
        minutes (int): number of minutes until self-destruct
        seconds (int): number of seconds until self-destruct
        """
        df_zip = f"{self.file_name}.zip"
        self_destruct_time = time.time() + hours * 60 * 60 + minutes * 60 + seconds
        while True:
            if time.time() > self_destruct_time:
                os.remove(df_zip)
                print(f"{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - Zip file deleted due to self-destruct timer.")
                break
            time.sleep(5)
            
# create a Zipndel object with password protection and self-destruct timer
zipper = Zipndel(file_name='test_file', file_format='csv', password='mypassword', self_destruct_time=(0, 3, 0))

# zip the dataframe with password protection and self-destruct timer
zipper.zipit(df)


2023-03-06 01:16:26 - Zip file deleted due to self-destruct timer.


Exception in thread Thread-10:
Traceback (most recent call last):
  File "/Users/m0/mambaforge/envs/zenv/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/Users/m0/mambaforge/envs/zenv/lib/python3.9/threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/93/6sy7vf8142969b_8w873sxcc0000gn/T/ipykernel_25294/936492472.py", line 72, in self_destruct
FileNotFoundError: [Errno 2] No such file or directory: 'test_file.zip'


In [ ]:
import os
import pyzipper
import pandas as pd
from getpass import getpass
import time


class Zipndel:
    def __init__(self, file_name, file_format, self_destruct_time):
        self.file_name = file_name
        self.file_format = file_format
        self.self_destruct_time = self_destruct_time
        self.password = None

        # prompt user for password
        self.password = getpass(prompt='Enter password for zip file: ')

    def zipit(self, df):
        # write dataframe to csv file
        df.to_csv(f"{self.file_name}.{self.file_format}", index=False)

        # create a zip file and write the csv file to it
        with pyzipper.AESZipFile(f"{self.file_name}.zip",
                                 'w',
                                 compression=pyzipper.ZIP_LZMA,
                                 encryption=pyzipper.WZ_AES) as zf:
            zf.setpassword(self.password.encode('utf-8'))
            zf.write(f"{self.file_name}.{self.file_format}")

        # delete the csv file
        os.remove(f"{self.file_name}.{self.file_format}")

        # initiate self-destruct timer
        timer = self.self_destruct_time
        while timer != (0, 0, 0):
            print(f"Time remaining: {timer[0]} hours {timer[1]} minutes {timer[2]} seconds")
            timer = (timer[0], timer[1], timer[2]-1)
            time.sleep(1)

        # delete the zip file
        os.remove(f"{self.file_name}.zip")





In [15]:
Uzipndel must prompt the same passsword create in Zipndel. Implement Unzipndel porpeely

class Unzipndel:
    def __init__(self, file_name, password):
        self.file_name = file_name
        self.password = password

    def unzipit(self):
        # extract the zip file
        with pyzipper.AESZipFile(f"{self.file_name}.zip", 'r') as zf:
            zf.setpassword(self.password.encode('utf-8'))
            zf.extractall()

        # read the extracted file into a pandas dataframe
        df = pd.read_csv(f"{self.file_name}.csv")

        # delete the extracted file
        os.remove(f"{self.file_name}.csv")

        return df

NameError: name 'Uzipndelme' is not defined

In [19]:
class Unzipndel:
    def __init__(self, file_name):
        self.file_name = file_name

xx

Enter password for zip file:  ········


FileNotFoundError: [Errno 2] No such file or directory: 'test_file.csv'

2023-03-06 01:13:46 - Zip file deleted due to self-destruct timer.


Enter password for zip file:  ········


FileNotFoundError: [Errno 2] No such file or directory: 'test_file.zip'

In [ ]:
# create an Unzipndel object
unzipper = Unzipndel(file_name='test_file')

# extract the dataframe from the zip file
df = unzipper.unzipit()

# display the dataframe
print(df)

In [20]:
import io
import os
import pyzipper
import pandas as pd
from getpass import getpass


class Unzipndel:
    def __init__(self, file_name):
        self.file_name = file_name
    
    def unzipit(self):
        # prompt user for password
        password = getpass(prompt='Enter password for zip file: ')

        # read the csv file from the zip archive
        with pyzipper.AESZipFile(f"{self.file_name}.zip") as zf:
            try:
                zf.setpassword(password.encode('utf-8'))
                with zf.open(f"{self.file_name}.csv") as file:
                    df = pd.read_csv(io.BytesIO(file.read()))
            except RuntimeError:
                raise ValueError('Incorrect password')

        return df
# create an Unzipndel object
unzipper = Unzipndel(file_name='test_file')

# extract the dataframe from the zip file
df = unzipper.unzipit()

# display the dataframe
print(df)

Enter password for zip file:  ········


FileNotFoundError: [Errno 2] No such file or directory: 'test_file.zip'

In [22]:
# create an Unzipndel object
unzipper = Unzipndel(file_name='test_file')

# extract the dataframe from the zip file
df = unzipper.unzipit()

# display the dataframe
print(df)

Enter password for zip file:  ········


FileNotFoundError: [Errno 2] No such file or directory: 'test_file.zip'

In [23]:
# create a dataframe
df = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6], 'C': [7, 8, 9]})

# create a Zipndel object with self-destruct timer
zipper = Zipndel(file_name='test_file', file_format='csv', self_destruct_time=(0, 5, 0))

# zip the dataframe with password protection and self-destruct timer
zipper.zipit(df)


Enter password for zip file:  ········


In [24]:
import io
import os
import pyzipper
import pandas as pd
from getpass import getpass


class Unzipndel:
    def __init__(self, file_name):
        self.file_name = file_name
    
    def unzipit(self):
        # prompt user for password
        password = getpass(prompt='Enter password for zip file: ')

        # read the csv file from the zip archive
        with pyzipper.AESZipFile(f"{self.file_name}.zip") as zf:
            try:
                zf.setpassword(password.encode('utf-8'))
                with zf.open(f"{self.file_name}.csv") as file:
                    df = pd.read_csv(io.BytesIO(file.read()))
            except RuntimeError:
                raise ValueError('Incorrect password')

        return df

# create an Unzipndel object
unzipper = Unzipndel(file_name='test_file')

# extract the dataframe from the zip file
df = unzipper.unzipit()

# display the dataframe
print(df)

Enter password for zip file:  ········


KeyError: "There is no item named 'test_file.csv' in the archive"

In [25]:
import ioff
import os
import pyzipper
import pandas as pd
from getpass import getpass
from datetime import datetime, timedelta


class Zipndel:
    def __init__(self, file_name, file_format='csv', password=None, self_destruct_time=None):
        self.file_name = file_name
        self.file_format = file_format
        self.password = password
        self.self_destruct_time = self_destruct_time
    
    def zipit(self, df):
        # write the dataframe to a csv file
        csv_data = df.to_csv(index=False)
        
        # create the zip archive
        with pyzipper.AESZipFile(f"{self.file_name}.zip", 'w', compression=pyzipper.ZIP_DEFLATED) as zf:
            # set password for the zip archive
            if self.password:
                zf.setpassword(self.password.encode('utf-8'))
            
            # add the csv file to the archive
            zf.writestr(f"{self.file_name}.csv", csv_data)
            
            # set self-destruct timer for the zip archive
            if self.self_destruct_time:
                time_now = datetime.now()
                self_destruct_time = time_now + timedelta(hours=self.self_destruct_time[0],
                                                          minutes=self.self_destruct_time[1],
                                                          seconds=self.self_destruct_time[2])
                zf.comment = self_destruct_time.strftime('%Y-%m-%d %H:%M:%S').encode('utf-8')
        
        return f"{self.file_name}.zip"


class Unzipndel:
    def __init__(self, file_name, password):
        self.file_name = file_name
        self.password = password
    
    def unzipit(self):
        # extract csv file from the zip archive to a pandas dataframe
        with pyzipper.AESZipFile(f"{self.file_name}.zip") as zf:
            try:
                zf.setpassword(self.password.encode('utf-8'))
                with zf.open(f"{self.file_name}.csv") as file:
                    df = pd.read_csv(io.BytesIO(file.read()))
            except RuntimeError:
                raise ValueError('Incorrect password')
        
        return df
    
    
# create a dataframe
df = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6], 'C': [7, 8, 9]})

# create a Zipndel object with password protection and self-destruct timer
zipper = Zipndel(file_name='test_file', file_format='csv', password='mypassword', self_destruct_time=(0, 5, 0))

# zip the dataframe with password protection and self-destruct timer
zip_file = zipper.zipit(df)



AttributeError: 'NoneType' object has no attribute 'update_zipinfo'

Exception in thread Thread-11:
Traceback (most recent call last):
  File "/Users/m0/mambaforge/envs/zenv/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/Users/m0/mambaforge/envs/zenv/lib/python3.9/threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/93/6sy7vf8142969b_8w873sxcc0000gn/T/ipykernel_25294/936492472.py", line 73, in self_destruct
AttributeError: type object 'datetime.datetime' has no attribute 'datetime'


In [ ]:
# create an Unzipndel object
unzipper = Unzipndel(file_name='test_file', password='mypassword')

# extract the dataframe from the zip file
df = unzipper.unzipit()

# display the dataframe
print(df)
